In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import pandas as pd
prolongations = pd.read_csv('prolongations.csv')
print(prolongations.head())

    id        month                             AM
0   42  ноябрь 2022   Васильев Артем Александрович
1  453  ноябрь 2022   Васильев Артем Александрович
2  548  ноябрь 2022      Михайлов Андрей Сергеевич
3   87  ноябрь 2022  Соколова Анастасия Викторовна
4  429  ноябрь 2022  Соколова Анастасия Викторовна


In [3]:
financial_data = pd.read_csv('financial_data.csv')
print(financial_data.head())

    id        Причина дубля Ноябрь 2022 Декабрь 2022 Январь 2023 Февраль 2023  \
0   42                  NaN   36 220,00          NaN         NaN          NaN   
1  657  первая часть оплаты        стоп          NaN         NaN          NaN   
2  657  вторая часть оплаты        стоп          NaN         NaN          NaN   
3  594                  NaN        стоп          NaN         NaN          NaN   
4  665                  NaN   10 000,00          NaN         NaN          NaN   

  Март 2023 Апрель 2023 Май 2023 Июнь 2023 Июль 2023 Август 2023  \
0       NaN         NaN      NaN       NaN       NaN         NaN   
1       NaN         NaN      NaN       NaN       NaN         NaN   
2       NaN         NaN      NaN       NaN       NaN         NaN   
3       NaN         NaN      NaN       NaN       NaN         NaN   
4       NaN         NaN      NaN       NaN       NaN         NaN   

  Сентябрь 2023 Октябрь 2023 Ноябрь 2023 Декабрь 2023 Январь 2024  \
0           NaN          NaN       

In [4]:
month_columns = financial_data.columns[2:-1].tolist()  # все колонки начиная с 'Ноябрь 2022'


In [5]:
# Функция для реобразование данных
def parse_financial_value(value):
    if pd.isna(value):
        return np.nan
    elif value in ['стоп', 'end']:
        return np.nan
    elif value == 'в ноль':
        return 0.0
    else:
        value = value.replace(' ', '').replace('\xa0', '').replace(',', '.')
        return float(value)


In [6]:
def clean_financial_data(financial_df, month_cols):
    # Копируем данные
    financial_clean = financial_df.copy()
    
    # Преобразуем все данные
    
    for month_col in month_cols:
        financial_clean[month_col] = financial_clean[month_col].apply(parse_financial_value)

    # Суммируем дубли
    
    financial_aggregated = financial_clean.groupby('id', as_index=False).agg({
        **{col: 'sum' for col in month_cols},
        'Account': 'first',
        'Причина дубля': lambda x: ', '.join(x.dropna().astype(str)) # через запятую записываем причины дублей
    })
    return financial_aggregated

In [7]:
def merge_data_with_priority(prolongations_df, financial_clean_df, month_cols):


    # Создаем объединенную таблицу с учетом того, что данные в prolongations.csv являются первичными по отношению к financial_data
    
    merged_data = pd.merge(
        prolongations_df[['id', 'month', 'AM']],
        financial_clean_df[['id'] + month_cols],
        on='id',
        how='inner'  # только проекты, которые есть в обоих файлах
    )
    
    return merged_data


def calculate_coefficients_for_month(target_month, month_list, merged_data):
    """
    Рассчитывает коэффициенты K1 и K2 для конкретного месяца
    с учетом ПРИОРИТЕТА данных из prolongations
    """
    month_index = month_list.index(target_month)
    
    # Проверяем, что есть достаточно предыдущих месяцев
    if month_index < 2:
        return None, None
    
    prev_month_1 = month_list[month_index - 1]  # первый предыдущий месяц
    prev_month_2 = month_list[month_index - 2]  # второй предыдущий месяц

    # вычисляем коэф 1
    
    # Находим проекты, которые завершились в prev_month_1 (согласно prolongations)
    projects_ended_prev1 = merged_data[
        merged_data['month'].str.lower() == prev_month_1.lower()
    ]
    
    k1 = 0
    k1_details = {'numerator': 0, 'denominator': 0, 'projects_count': 0}
    
    if not projects_ended_prev1.empty:
        # отгрузка в последний месяц реализации (prev_month_1)
        denominator_k1 = projects_ended_prev1[prev_month_1].fillna(0).sum()
        
        # отгрузка в ыелевом месяце
        # Фильтруем проекты, у которых есть отгрузка в целевом месяце
        prolonged_projects = projects_ended_prev1[projects_ended_prev1[target_month].fillna(0) > 0]
        numerator_k1 = prolonged_projects[target_month].sum()
        
        # Вычисляем коэффициент
        k1 = numerator_k1 / denominator_k1 if denominator_k1 > 0 else 0
        
        k1_details = {
            'numerator': numerator_k1,
            'denominator': denominator_k1,
            'projects_count': len(prolonged_projects)
        }
        
        print(f"      K1: {numerator_k1:.0f} / {denominator_k1:.0f} = {k1:.1%}")

    # вычисляем коэф 2
    
    # Находим проекты, которые завершились в prev_month_2 (согласно prolongations)
    projects_ended_prev2 = merged_data[
        merged_data['month'].str.lower() == prev_month_2.lower()
    ]
    
    k2 = 0
    k2_details = {'numerator': 0, 'denominator': 0, 'projects_count': 0}
    
    if not projects_ended_prev2.empty:
        # проекты, которые не были пролонгированы в первый месяц
        no_prolong_prev1 = projects_ended_prev2[projects_ended_prev2[prev_month_1].fillna(0) == 0]
        
        # знаменатель K2: отгрузка в prev_month_2
        denominator_k2 = no_prolong_prev1[prev_month_2].fillna(0).sum()
        
        # числитель K2: отгрузка в ЦЕЛЕВОМ месяце
        prolonged_second = no_prolong_prev1[no_prolong_prev1[target_month].fillna(0) > 0]
        numerator_k2 = prolonged_second[target_month].sum()
        
        # Вычисляем коэффициент
        k2 = numerator_k2 / denominator_k2 if denominator_k2 > 0 else 0
        
        k2_details = {
            'numerator': numerator_k2,
            'denominator': denominator_k2,
            'projects_count': len(prolonged_second)
        }
        
        print(f"      K2: {numerator_k2:.0f} / {denominator_k2:.0f} = {k2:.1%}")

    return (k1, k1_details), (k2, k2_details)

def analyze_all_months(month_list, prolongations_df, financial_df):
    
    # Очищаем финансовые данные
    financial_clean = clean_financial_data(financial_df, month_list)
    
    # Объединяем с приоритетом prolongations
    merged_data = merge_data_with_priority(prolongations_df, financial_clean, month_list)
    
    # Рассчитываем коэффициенты для каждого месяца
    all_results = []
    
    for i, month in enumerate(month_list):
        if i < 2:
            continue
            
        
        k1_data, k2_data = calculate_coefficients_for_month(month, month_list, merged_data)
        
        if k1_data is not None and k2_data is not None:
            k1, k1_details = k1_data
            k2, k2_details = k2_data
            
            all_results.append({
                'month': month,
                'k1': k1,
                'k2': k2,
                'k1_numerator': k1_details['numerator'],
                'k1_denominator': k1_details['denominator'],
                'k2_numerator': k2_details['numerator'],
                'k2_denominator': k2_details['denominator'],
                'k1_projects': k1_details['projects_count'],
                'k2_projects': k2_details['projects_count']
            })
    
    return pd.DataFrame(all_results)

def create_manager_report(results_df, merged_data):
    """
    Создает отчет по менеджерам
    """

    
    report = results_df.copy()
    
    # Форматируем проценты
    report['K1_пролонгация'] = (report['k1'] * 100).round(1).astype(str) + '%'
    report['K2_пролонгация'] = (report['k2'] * 100).round(1).astype(str) + '%'
    
    # Округляем числа
    for col in ['k1_numerator', 'k1_denominator', 'k2_numerator', 'k2_denominator']:
        report[col] = report[col].round(0).astype(int)
    
    final_report = report[[
        'month', 'K1_пролонгация', 'K2_пролонгация',
        'k1_numerator', 'k1_denominator', 'k1_projects',
        'k2_numerator', 'k2_denominator', 'k2_projects'
    ]].rename(columns={
        'month': 'Месяц',
        'k1_numerator': 'K1_числитель',
        'k1_denominator': 'K1_знаменатель', 
        'k1_projects': 'K1_кол-во_проектов',
        'k2_numerator': 'K2_числитель',
        'k2_denominator': 'K2_знаменатель',
        'k2_projects': 'K2_кол-во_проектов'
    })
    
    return final_report

In [8]:
def calculate_manager_coefficients(target_month, month_list, merged_data):
    """
    Рассчитывает коэффициенты K1 и K2 для каждого менеджера в конкретном месяце
    """
    month_index = month_list.index(target_month)
    
    if month_index < 2:
        return pd.DataFrame()
    
    prev_month_1 = month_list[month_index - 1]
    prev_month_2 = month_list[month_index - 2]
    
    manager_results = []
    all_managers = merged_data['AM'].unique()
    
    for manager in all_managers:
        # Фильтруем данные по менеджеру
        manager_data = merged_data[merged_data['AM'] == manager]
        
        # КОЭФФИЦИЕНТ 1 для менеджера
        projects_ended_prev1 = manager_data[
            manager_data['month'].str.lower() == prev_month_1.lower()
        ]
        
        k1_numerator, k1_denominator, k1_projects = 0, 0, 0
        k2_numerator, k2_denominator, k2_projects = 0, 0, 0
        
        if not projects_ended_prev1.empty:
            # K1 расчет
            k1_denominator = projects_ended_prev1[prev_month_1].fillna(0).sum()
            prolonged_k1 = projects_ended_prev1[projects_ended_prev1[target_month].fillna(0) > 0]
            k1_numerator = prolonged_k1[target_month].sum()
            k1_projects = len(prolonged_k1)
        
        # КОЭФФИЦИЕНТ 2 для менеджера
        projects_ended_prev2 = manager_data[
            manager_data['month'].str.lower() == prev_month_2.lower()
        ]
        
        if not projects_ended_prev2.empty:
            # K2 расчет
            no_prolong_prev1 = projects_ended_prev2[projects_ended_prev2[prev_month_1].fillna(0) == 0]
            k2_denominator = no_prolong_prev1[prev_month_2].fillna(0).sum()
            prolonged_k2 = no_prolong_prev1[no_prolong_prev1[target_month].fillna(0) > 0]
            k2_numerator = prolonged_k2[target_month].sum()
            k2_projects = len(prolonged_k2)
        
        # Рассчитываем коэффициенты
        k1 = k1_numerator / k1_denominator if k1_denominator > 0 else 0
        k2 = k2_numerator / k2_denominator if k2_denominator > 0 else 0
        
        # Общее количество проектов менеджера
        total_projects = len(manager_data)
        
        manager_results.append({
            'manager': manager,
            'month': target_month,
            'k1': k1,
            'k2': k2,
            'k1_numerator': k1_numerator,
            'k1_denominator': k1_denominator,
            'k2_numerator': k2_numerator,
            'k2_denominator': k2_denominator,
            'k1_projects': k1_projects,
            'k2_projects': k2_projects,
            'total_projects': total_projects
        })
    
    return pd.DataFrame(manager_results)

def analyze_all_months_with_managers(month_list, prolongations_df, financial_df):
    """
    Анализирует все месяцы с детализацией по менеджерам
    """    
    financial_clean = clean_financial_data(financial_df, month_list)
    merged_data = merge_data_with_priority(prolongations_df, financial_clean, month_list)
    
    all_manager_results = []
    all_department_results = []
    
    for i, month in enumerate(month_list):
        if i < 2:
            continue
        
        # Расчет по менеджерам
        manager_monthly = calculate_manager_coefficients(month, month_list, merged_data)
        if not manager_monthly.empty:
            all_manager_results.append(manager_monthly)
        
        # Расчет по отделу 
        department_result = calculate_department_coefficients(month, month_list, merged_data)
        if department_result:
            all_department_results.append(department_result)
    
    manager_df = pd.concat(all_manager_results, ignore_index=True) if all_manager_results else pd.DataFrame()
    department_df = pd.DataFrame(all_department_results) if all_department_results else pd.DataFrame()
    
    return manager_df, department_df

def calculate_department_coefficients(target_month, month_list, merged_data):
    """
    Рассчитывает коэффициенты по всему отделу
    """
    month_index = month_list.index(target_month)
    
    if month_index < 2:
        return None
    
    prev_month_1 = month_list[month_index - 1]
    prev_month_2 = month_list[month_index - 2]
    
    # K1 по отделу
    projects_ended_prev1 = merged_data[
        merged_data['month'].str.lower() == prev_month_1.lower()
    ]
    
    k1_numerator, k1_denominator, k1_projects = 0, 0, 0
    k2_numerator, k2_denominator, k2_projects = 0, 0, 0
    
    if not projects_ended_prev1.empty:
        k1_denominator = projects_ended_prev1[prev_month_1].fillna(0).sum()
        prolonged_k1 = projects_ended_prev1[projects_ended_prev1[target_month].fillna(0) > 0]
        k1_numerator = prolonged_k1[target_month].sum()
        k1_projects = len(prolonged_k1)
    
    # K2 по отделу
    projects_ended_prev2 = merged_data[
        merged_data['month'].str.lower() == prev_month_2.lower()
    ]
    
    if not projects_ended_prev2.empty:
        no_prolong_prev1 = projects_ended_prev2[projects_ended_prev2[prev_month_1].fillna(0) == 0]
        k2_denominator = no_prolong_prev1[prev_month_2].fillna(0).sum()
        prolonged_k2 = no_prolong_prev1[no_prolong_prev1[target_month].fillna(0) > 0]
        k2_numerator = prolonged_k2[target_month].sum()
        k2_projects = len(prolonged_k2)
    
    k1 = k1_numerator / k1_denominator if k1_denominator > 0 else 0
    k2 = k2_numerator / k2_denominator if k2_denominator > 0 else 0
    
    return {
        'month': target_month,
        'k1': k1,
        'k2': k2,
        'k1_numerator': k1_numerator,
        'k1_denominator': k1_denominator,
        'k2_numerator': k2_numerator,
        'k2_denominator': k2_denominator,
        'k1_projects': k1_projects,
        'k2_projects': k2_projects
    }


In [9]:
def create_manager_monthly_report(manager_df):
    """
    Создает помесячный отчет по менеджерам
    """
    
    if manager_df.empty:
        return pd.DataFrame()
    
    report = manager_df.copy()
    report['K1_пролонгация'] = (report['k1'] * 100).round(1).astype(str) + '%'
    report['K2_пролонгация'] = (report['k2'] * 100).round(1).astype(str) + '%'
    
    # Округляем числа
    for col in ['k1_numerator', 'k1_denominator', 'k2_numerator', 'k2_denominator']:
        report[col] = report[col].round(0).astype(int)
    
    final_report = report[[
        'manager', 'month', 'K1_пролонгация', 'K2_пролонгация',
        'k1_numerator', 'k1_denominator', 'k1_projects',
        'k2_numerator', 'k2_denominator', 'k2_projects',
        'total_projects'
    ]].rename(columns={
        'manager': 'Менеджер',
        'month': 'Месяц',
        'k1_numerator': 'K1_числитель',
        'k1_denominator': 'K1_знаменатель', 
        'k1_projects': 'K1_кол-во_проектов',
        'k2_numerator': 'K2_числитель',
        'k2_denominator': 'K2_знаменатель',
        'k2_projects': 'K2_кол-во_проектов',
        'total_projects': 'Всего_проектов'
    })
    
    return final_report

def create_manager_yearly_report(manager_df):
    """
    Создает годовой отчет по менеджерам
    """
    
    if manager_df.empty:
        return pd.DataFrame()
    
    # Группируем по менеджерам и суммируем за год
    yearly_report = manager_df.groupby('manager').agg({
        'k1_numerator': 'sum',
        'k1_denominator': 'sum',
        'k2_numerator': 'sum',
        'k2_denominator': 'sum',
        'k1_projects': 'sum',
        'k2_projects': 'sum',
        'total_projects': 'first'  
    }).reset_index()
    
    # Рассчитываем годовые коэффициенты
    yearly_report['k1'] = yearly_report.apply(
        lambda x: x['k1_numerator'] / x['k1_denominator'] if x['k1_denominator'] > 0 else 0, 
        axis=1
    )
    yearly_report['k2'] = yearly_report.apply(
        lambda x: x['k2_numerator'] / x['k2_denominator'] if x['k2_denominator'] > 0 else 0, 
        axis=1
    )
    
    yearly_report['K1_годовой'] = (yearly_report['k1'] * 100).round(1).astype(str) + '%'
    yearly_report['K2_годовой'] = (yearly_report['k2'] * 100).round(1).astype(str) + '%'
    
    final_report = yearly_report[[
        'manager', 'K1_годовой', 'K2_годовой',
        'k1_numerator', 'k1_denominator', 'k1_projects',
        'k2_numerator', 'k2_denominator', 'k2_projects',
        'total_projects'
    ]].rename(columns={
        'manager': 'Менеджер',
        'k1_numerator': 'K1_числитель_год',
        'k1_denominator': 'K1_знаменатель_год', 
        'k1_projects': 'K1_проектов_год',
        'k2_numerator': 'K2_числитель_год',
        'k2_denominator': 'K2_знаменатель_год',
        'k2_projects': 'K2_проектов_год',
        'total_projects': 'Всего_проектов'
    })
    
    return final_report

def create_department_reports(department_df):
    """
    Создает отчеты по отделу
    """    
    if department_df.empty:
        return pd.DataFrame(), pd.DataFrame()
    
    # Помесячный отчет по отделу
    monthly_report = department_df.copy()
    monthly_report['K1_пролонгация'] = (monthly_report['k1'] * 100).round(1).astype(str) + '%'
    monthly_report['K2_пролонгация'] = (monthly_report['k2'] * 100).round(1).astype(str) + '%'
    
    for col in ['k1_numerator', 'k1_denominator', 'k2_numerator', 'k2_denominator']:
        monthly_report[col] = monthly_report[col].round(0).astype(int)
    
    monthly_final = monthly_report[[
        'month', 'K1_пролонгация', 'K2_пролонгация',
        'k1_numerator', 'k1_denominator', 'k1_projects',
        'k2_numerator', 'k2_denominator', 'k2_projects'
    ]].rename(columns={
        'month': 'Месяц',
        'k1_numerator': 'K1_числитель',
        'k1_denominator': 'K1_знаменатель', 
        'k1_projects': 'K1_кол-во_проектов',
        'k2_numerator': 'K2_числитель',
        'k2_denominator': 'K2_знаменатель',
        'k2_projects': 'K2_кол-во_проектов'
    })
    
    # Годовой отчет по отделу
    yearly_k1 = department_df['k1_numerator'].sum() / department_df['k1_denominator'].sum() if department_df['k1_denominator'].sum() > 0 else 0
    yearly_k2 = department_df['k2_numerator'].sum() / department_df['k2_denominator'].sum() if department_df['k2_denominator'].sum() > 0 else 0
    
    yearly_report = pd.DataFrame({
        'Показатель': ['K1_годовой', 'K2_годовой'],
        'Значение': [f"{yearly_k1:.1%}", f"{yearly_k2:.1%}"],
        'Числитель': [department_df['k1_numerator'].sum(), department_df['k2_numerator'].sum()],
        'Знаменатель': [department_df['k1_denominator'].sum(), department_df['k2_denominator'].sum()],
        'Проектов': [department_df['k1_projects'].sum(), department_df['k2_projects'].sum()]
    })
    
    return monthly_final, yearly_report

In [10]:
def save_all_reports_to_excel(manager_monthly, manager_yearly, dept_monthly, dept_yearly):
    """
    Сохраняет все отчеты в один Excel файл с разными листами
    """
    
    # Создаем Excel файл с несколькими листами
    with pd.ExcelWriter('полный_отчет_пролонгации.xlsx', engine='openpyxl') as writer:
        
        # Сохраняем каждый отчет на отдельный лист
        if not manager_monthly.empty:
            manager_monthly.to_excel(writer, sheet_name='Менеджеры_помесячно', index=False)
        
        if not manager_yearly.empty:
            manager_yearly.to_excel(writer, sheet_name='Менеджеры_годовые', index=False)
        
        if not dept_monthly.empty:
            dept_monthly.to_excel(writer, sheet_name='Отдел_помесячно', index=False)
        
        if not dept_yearly.empty:
            dept_yearly.to_excel(writer, sheet_name='Отдел_годовые', index=False)


In [11]:
# Запускаем анализ
manager_df, department_df = analyze_all_months_with_managers(month_columns, prolongations, financial_data)


# Создаем все отчеты
manager_monthly = create_manager_monthly_report(manager_df)
manager_yearly = create_manager_yearly_report(manager_df)
dept_monthly, dept_yearly = create_department_reports(department_df)

# Сохраняем все отчеты в CSV
save_all_reports_to_excel(manager_monthly, manager_yearly, dept_monthly, dept_yearly)